In [1]:
# Start with loading all necessary libraries
import pandas as pd
import pyodbc
from collections import Counter
import re

In [2]:
sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=L_AAGDATIX;'
                            'DATABASE=DatixCRM;'
                            'Trusted_Connection=yes') 
query = "set transaction isolation level read uncommitted select b.description, c.description, a.inc_notes, a.inc_actiontaken from DatixCRM.dbo.code_unit b join DatixCRM.dbo.incidents_main a on a.inc_unit = b.code join DatixCRM.dbo.code_locactual c on a.inc_locactual = c.code where a.inc_type='PAT' and a.inc_category='MEDIC' and a.inc_organisation='QA'"
df = pd.read_sql(query, sql_conn)
column_names = list(df.columns)
column_names[0] = 'division_care_group'
column_names[1] = 'location'
df.columns = column_names
df.head()

,division_care_group,location,inc_notes,inc_actiontaken
0,Medicine & Urgent Care Division - Medicine,E6/7,on drug rounds found Amlodipine not given for ...,ask Doctor to review which was done and was st...
1,Medicine & Urgent Care Division - Medicine,C5,HAEMOPHILIA PATIENT\r\n\r\nPATIENT NOT ADMINIS...,PATIENT CONTACTED HAEMOPHILIA CNS ON MORNING O...
2,Medicine and Urgent Care Division - Urgent Care,Emergency Department (ED),"Resus controlled order booked went missing, ha...",reported to ED Pharmacist
3,Medicine & Urgent Care Division - Medicine,C7,MIDAZOLAM SIGNED OUT OF CD BOOK FOR PATIENT . ...,CURRENTLY UNDER INVESTIGATION . NO HARM TO PAT...
4,Surgical and Outpatients Division - MSK / Head...,D5,I was giving the oral medication(CD-oxycodone)...,I reported the situation to the nurse in charg...


In [3]:
#consecutive_caps_series_location = df['location'].str.findall(r"((?:\b[A-Za-z&]+\b\s)+\([A-Za-z][A-Za-z]+\))")
consecutive_caps_series_notes = df['inc_notes'].str.findall(r"((?:\b[A-Za-z]+\b\s)+\([A-Za-z][A-Za-z]+\))")
consecutive_caps_series_action = df['inc_actiontaken'].str.findall(r"((?:\b[A-Za-z]+\b\s)+\([A-Za-z][A-Za-z]+\))")
#consecutive_caps_series = pd.concat([consecutive_caps_series_location,consecutive_caps_series_notes,consecutive_caps_series_action])
consecutive_caps_series = pd.concat([consecutive_caps_series_notes,consecutive_caps_series_action])
print(consecutive_caps_series)
consecutive_caps_series = consecutive_caps_series[consecutive_caps_series.map(lambda d: len(d)) > 0]
slist = []
for x in consecutive_caps_series:
    slist.extend(x)

term_to_abbreviation_dict = {}
for l in slist:
    inside_brackets = re.findall(r"\(([A-Za-z]+)\)", l)[0]
    len_inside_brackets = len(inside_brackets)
    num_words = len(l.split()) - 1
    words_before_brackets = []
    for i in range(len_inside_brackets):
        if i<num_words: words_before_brackets.insert(0,l.split(" ")[-i-2])
    string_before_brackets = " ".join(words_before_brackets)
    if string_before_brackets[0].lower()==inside_brackets[0].lower(): 
        term_to_abbreviation_dict[string_before_brackets] = inside_brackets
for key,val in dict(term_to_abbreviation_dict).items():
    if val=='OD': del term_to_abbreviation_dict[key]
    elif val=='PIVOTAL': del term_to_abbreviation_dict[key]
    elif val.lower().startswith('pri'): del term_to_abbreviation_dict[key]
    elif val=='fresh': del term_to_abbreviation_dict[key]
    elif val=='West': del term_to_abbreviation_dict[key]
    elif val.lower()=='oxynorm': del term_to_abbreviation_dict[key]
    elif val=='methylprednisolone': del term_to_abbreviation_dict[key]
    elif val=='cetraben': del term_to_abbreviation_dict[key]
    elif val=='Levemir': del term_to_abbreviation_dict[key]
    elif val=='Desmopressin': del term_to_abbreviation_dict[key]
    elif val.lower()=='oramorph': del term_to_abbreviation_dict[key]
    elif val=='insulatard': del term_to_abbreviation_dict[key]
    elif val=='missing': del term_to_abbreviation_dict[key]
    elif val=='insulatard': del term_to_abbreviation_dict[key]
    elif val=='SS': del term_to_abbreviation_dict[key]
    elif val=='Tramadol': del term_to_abbreviation_dict[key]
    elif val.lower()=='eprex': del term_to_abbreviation_dict[key]
    elif val=='Tuesday': del term_to_abbreviation_dict[key]
    elif val=='cloudy': del term_to_abbreviation_dict[key]
    elif val=='stable': del term_to_abbreviation_dict[key]
    elif val=='Solent': del term_to_abbreviation_dict[key]
    elif val=='carer': del term_to_abbreviation_dict[key]
    elif val=='Omeprazole': del term_to_abbreviation_dict[key]
term_to_abbreviation_dict['Intravenous'] = term_to_abbreviation_dict.pop('Intravenous Antibiotics')
term_to_abbreviation_dict['Morphine sulphate'] = term_to_abbreviation_dict.pop('Morphine sulphate MR')
#term_to_abbreviation_dict['Department of Critical Care'] = 'DCCQ'
#term_to_abbreviation_dict['MAU'] = 'AMU'

consecutive_caps_series_notes_inv = df['inc_notes'].str.findall(r"[A-Za-z][A-Za-z]+\s\((?:\b[A-Za-z]+\b\s)(?:\b[A-Za-z]+\b\s?)+\)")
consecutive_caps_series_action_inv = df['inc_actiontaken'].str.findall(r"[A-Za-z][A-Za-z]+\s\((?:\b[A-Za-z]+\b\s)(?:\b[A-Za-z]+\b\s?)+\)")
consecutive_caps_series_notes_inv = consecutive_caps_series_notes_inv[consecutive_caps_series_notes_inv.map(lambda d: len(d)) > 0]
slist = []
for x in consecutive_caps_series_notes_inv:
    slist.extend(x)
for l in slist:
    inside_brackets = re.findall(r"\((?:\b[A-Za-z]+\b\s)(?:\b[A-Za-z]+\b\s?)+\)", l)[0]
    words_inside_brackets = len(inside_brackets.split())
    word_before_brackets = l.split()[0]
    if words_inside_brackets==len(word_before_brackets) and inside_brackets[1].lower()==word_before_brackets[0].lower() and inside_brackets.split()[1].lower()[0]==word_before_brackets[1].lower():
        term_to_abbreviation_dict[inside_brackets[1:-1]] = word_before_brackets
if 'CARDIAC DAY UNIT' in term_to_abbreviation_dict.keys(): del term_to_abbreviation_dict['CARDIAC DAY UNIT']
#term_to_abbreviation_dict = {key.lower():val.lower() for (key,val) in term_to_abbreviation_dict.items()}
for key,val in term_to_abbreviation_dict.items():
    print(key+" & "+val+'\\\\')

0       []
1       []
2       []
3       []
4       []
5       []
6       []
7       []
8       []
9       []
10      []
11      []
12      []
13      []
14      []
15      []
16      []
17      []
18      []
19      []
20      []
21      []
22      []
23      []
24      []
25      []
26      []
27      []
28      []
29      []
        ..
9533    []
9534    []
9535    []
9536    []
9537    []
9538    []
9539    []
9540    []
9541    []
9542    []
9543    []
9544    []
9545    []
9546    []
9547    []
9548    []
9549    []
9550    []
9551    []
9552    []
9553    []
9554    []
9555    []
9556    []
9557    []
9558    []
9559    []
9560    []
9561    []
9562    []
Length: 19126, dtype: object
milligrams & mg\\
immediate need care form & ICNF\\
medication administration record & MAR\\
Electronic Discharge Summary & EDS\\
Mental Health Nurse & MHN\\
not medically fit for discharge & NMFFD\\
district nurse & DN\\
microgram & mcg\\
Home Oxygen Order Form & HOOF\\
Long Term Oxygen Therapy & L

In [6]:
ward_num_series = df['location'].str.findall(r"[A-G][0-9]\s.+")
ward_num_series = ward_num_series[ward_num_series.map(lambda d: len(d)) > 0]
slist = []
for x in ward_num_series:
    slist.extend(x)
ward_name_to_num_dict = {}
for l in slist:
    l_split = l.split()
    name = " ".join(l_split[1:])
    name = name.strip("- ")
    ward_name_to_num_dict[name.lower()] = l_split[0].lower()
ward_name_to_num_dict['dccq'] = 'e5'
for key,val in ward_name_to_num_dict.items():
    print(key+": "+val)

acute stroke ward: f4
private patient unit: g5
postnatal ward: b7
neuro rehab: f1
nicu: b9
starfish: a7
antenatal ward: b6
stroke rehabilitation: f3
endoscopy suites: g5
labour ward: b8
shipwreck: a8
moprs (opened 20/07/16): e4
mary rose ward: b5
trauma: d5
older persons medicine: d1
dccq: e5


In [21]:
df_19 = df[df['inc_notes'].str.lower().str.contains("medication administration record",regex=False)]
print(df_19['inc_notes'].values)

['Patient is usually on methotrexate 10mg once weekly on Mondays. This was prescribed by the patient\'s GP and is shown on the summary care record. He has recently been admitted to QA in September, and was discharged to Jubilee House, in which the dose of methotrexate has not been changed (maintained 10mg once weekly on Mondays). The patient was discharged from Jubilee House on 8th November  to a nursing home. On the medication administration record (MAR) sheet, the methotrexate dose was written up as "25mg tablets, four tablets taken on Monday". \r\n\r\nMr T is then re-admitted to QA on the 12th November, methotrexate 100mg once a week is prescribed on the drug chart as the doctor has transcribed it from the nursing home MAR sheet. \r\n\r\nPlease see photocopies.'
 'Patient discharged from ITU on clonidine with plan to wean off once agitation settled. Medication administration recorded as "3" (drug not available)from night of 9th January to this morning 11th January.\r\n\r\nPatient wa

In [17]:
import csv
csv_columns = ['Term','Abbreviation']
csv_file = "Names.csv"
with open('test.csv', 'w') as f:
    for key in term_to_abbreviation_dict.keys():
        f.write("%s,%s\n"%(key,term_to_abbreviation_dict[key]))